In [2]:
import pandas as pd
import numpy as np

In [5]:
#import data
suffix_no_nuclear = 'no-nuclear_all-generators_20k-new_name'
suffix_normal = 'normal'

gen = pd.read_csv(f'./good_model_inputs/inputs_gen_{suffix_no_nuclear}.csv')
load = pd.read_csv(f'./good_model_inputs/inputs_load_no-nuclear.csv')
rencf = pd.read_csv(f'./good_model_inputs/inputs_renewableCF.csv')
trans = pd.read_csv(f'./good_model_inputs/inputs_trans_no-nuclear.csv')

/home/emfreese/anaconda3/envs/conda_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#create a list of our regions
regions = set(load['r'])

In [4]:
#calculate regions where generation > load
for reg in regions:
    gn = np.zeros((8759,))
    gn += gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] != 'Solar').values * (gen['FuelType'] != 'Wind').values, 'Capacity'].values.sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'Solar').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'solarCF'].values).sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'Wind').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'windCF'].values).sum()
    if (load.loc[load['r'] == reg, 'demandLoad'] > gn).values.sum() > 0:
        print(f'Generation cannot match load in region {reg}')
        print(max(load.loc[load['r'] == reg, 'demandLoad'] - gn))
    

Generation cannot match load in region NY_Z_GI
408.384
Generation cannot match load in region NY_Z_J
1807.0


In [7]:
#find the total generation for the region where generation > load

region = 'ERC_REST'
t = 5080

print( 'Nuclear generation ' + str(gen.loc[gen['RegionName'] == region].loc[gen['FuelType'] == 'Nuclear','Capacity'].sum()))

print( 'Total generation ' + str(gen.loc[gen['RegionName'] == region,'Capacity'].sum()))

#look at the transmission into the region where generation > load

print( 'Transmission into the region ' + str(trans.loc[(trans['r2'] == region).values * (trans['transCap'] > 1).values, 'transCap'].sum()*.972))

print( 'Transmission out of the region ' + str(trans.loc[(trans['r1'] == region).values * (trans['transCap'] > 1).values, 'transCap'].sum()))

#find the total amount of energy needed to/from that region if all used

print( f'Maximum net energy left after use from the region at time {t} ' + 
    str(trans.loc[(trans['r2'] == region).values, 'transCap'].sum()*.972 + 
       gen.loc[gen['RegionName'] == region,'Capacity'].sum() - 
        trans.loc[(trans['r1'] == region).values, 'transCap'].sum() - 
       load.loc[(load['r'] == 'NY_Z_GI')].loc[(load['t'] == t)]['demandLoad'].values))

Nuclear generation 0.0
Total generation 69205.1
Transmission into the region 11916.72
Transmission out of the region 5529
Maximum net energy left after use from the region at time 5080 [72968.832]


In [12]:
#find what time stamps are associated with demand > load 
#(we will go back and manually change our load based on this, assuming some of the region blacks out)
load.loc[load['r'] == region].loc[load['t'].isin(range(5001,6000))].loc[load['demandLoad'] >= gen.loc[gen['RegionName'] == region,'Capacity'].sum()]

,Unnamed: 0,r,t,demandLoad
5076,5076,ERC_REST,5077,70530.65
5077,5077,ERC_REST,5078,71687.55
5078,5078,ERC_REST,5079,72176.30
5079,5079,ERC_REST,5080,71879.60
5080,5080,ERC_REST,5081,70547.90
5099,5099,ERC_REST,5100,69603.75
5100,5100,ERC_REST,5101,72053.25
5101,5101,ERC_REST,5102,73046.85
5102,5102,ERC_REST,5103,73306.75
5103,5103,ERC_REST,5104,72714.50
